In [2]:
# Imports 
import os
import matplotlib.pyplot as plt
from array import array
from ROOT import TFile, TTree
import numpy as np
plt.rc('text', usetex=True)
import ROOT
import datetime

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-bpqoyaxl because the default path (/home/jovyan/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


Welcome to JupyROOT 6.22/06


In [3]:
def get_dirs(PROJECT_NAME,MLmethod):
    pathtowork="/work/clas12/users/gmat/scipio/projects"
    pathtovolatile="/volatile/clas12/users/gmat/clas12analysis.sidis.data/rga/ML/projects"
    dirs = [d for d in os.listdir(f"{pathtovolatile}/{PROJECT_NAME}/{MLmethod}/postprocess_binned")]
    bindirs = []
    for d in dirs:
        if ("bru" in d or "sdbnd" in d):
            continue
        else:
            bindirs.append(f"{pathtovolatile}/{PROJECT_NAME}/{MLmethod}/postprocess_binned/{d}")
    outdir=f"{pathtowork}/{PROJECT_NAME}/{MLmethod}/binnedPlots"
    return bindirs,outdir

In [11]:
pltfmt={
#     "isGoodEventWithoutML" : {"min": 0, "max": 1, "bins": 10, "xtitle": "isGoodEventWithoutML"},
#     "run": {"min": 0 , "max": 1, "bins": 100, "xtitle": "run"},
#     "fgID": {"min": 0 , "max": 1, "bins": 100, "xtitle": "fgID"},
    "x": {"min": 0 , "max": 1, "bins": 100, "xtitle": "x_{B}"},
    "Q2": {"min": 0 , "max": 10, "bins": 100, "xtitle": "Q^{2} [GeV^{2}]"},
    "Mx": {"min": -1 , "max": 4, "bins": 100, "xtitle": "M_{miss} [GeV]"},
    "z": {"min": 0 , "max": 1, "bins": 100, "xtitle": "z"},
#     "hel": {"min": -2 , "max": 2, "bins": 10, "xtitle": "helicity"},
    "Mgg": {"min": 0 , "max": 0.4, "bins": 100, "xtitle": "M_{#gamma#gamma} [GeV]"},
    "Mh": {"min": 0 , "max": 2, "bins": 100, "xtitle": "M_{#pi^{0}#pi^{+}} [GeV]"},
    "phi_h": {"min": -np.pi , "max": np.pi, "bins": 100, "xtitle": "#phi_{h}"},
    "phi_R0": {"min": -np.pi , "max": np.pi, "bins": 100, "xtitle": "#phi_{R}"},
    "phi_R1": {"min": -np.pi , "max": np.pi, "bins": 100, "xtitle": "#phi_{R}"},
    "th": {"min": 0 , "max": np.pi, "bins": 100, "xtitle": "#theta_{COM}"},
    "prob_g1": {"min": 0 , "max": 1, "bins": 100, "xtitle": "p(#gamma_{1})"},
    "prob_g2": {"min": 0 , "max": 1, "bins": 100, "xtitle": "p(#gamma_{2})"}
}

In [4]:
def get_cut_from_datatype(datatypes):
    if(not isinstance(datatypes,list)):
        datatypes=[datatypes]
    cuts=[]
    for datatype in datatypes:
        if(datatype=="all" or datatype==""):
            cuts.append("")
        elif(datatype=="Fall2018_inbending"):
            cuts.append("run>=5032 && run<=5332")
        elif(datatype=="Fall2018_outbending"):
            cuts.append("run>=5333 && run<=5666")
        elif(datatype=="Spring2019_inbending"):
            cuts.append("run>=6616 && run<=6783")
        elif(datatype=="MC_inbending"):
            cuts.append("run==-11")
        elif(datatype=="MC_outbending"):
            cuts.append("run==11")
        else:
            print("ERROR in get_cut_from_datatype: Unknown datatype",datatype,"...setting no cut...")
            cuts.append("")
            
    if(len(datatypes)==1):
        return cuts[0]
    else:
        return cuts

['',
 'run>=5032 && run<=5332',
 'run>=5333 && run<=5666',
 'run>=6616 && run<=6783']

In [161]:
def get_sorted_plots(plots,skipEdges=False):
    # Create a new dict to store the sorted plots
    sorted_plots = []
    
    for plot in plots:
        sorted_plot={}
        # Get the sorted index of centers
        sorted_index = np.lexsort(tuple([plot["centers"][:,n] for n in range(plot["centers"].shape[1])]))
        if(skipEdges):
            ny = np.count_nonzero(plot["centers"][:,0]==plot["centers"][0][0])
            nx = int(len(plot["centers"])/ny)
            
            sorted_index=list(sorted_index)
            del sorted_index[::nx]
            del sorted_index[nx-2::nx-1]
            sorted_index=np.array(sorted_index)

        # Iterate through the keys in plots
        for key in plot.keys():
            if key == 'name':
                sorted_plot['name']=plot[key]
                continue
            if(key=="hists"):
                arr=[plot[key][si] for si in sorted_index]
                sorted_plot[key]=arr
            else:
                sorted_plot[key]=plot[key][sorted_index]
        sorted_plots.append(sorted_plot)
    
    return sorted_plots

In [10]:
def collect_many(input_dir,prefix,datatypes):
    
    nCuts=len(datatypes)
    
    name_list = []
    min_list = []
    max_list = []
    center_list = []
    hlist = []
    
    cuts = get_cut_from_datatype(datatypes)
    
    for file in os.listdir(f"{input_dir}/"):
        # Only analyze .root files
        if not file.endswith(".root"):
            continue
        if(not file.startswith(prefix)):
                continue
        
        # Open the .root file
        tfile = ROOT.TFile(f"{input_dir}/{file}","READ")
        print("Reading TFile",file)
        
        # Loop over all TTrees in the .root files
        treenames = [ttree.GetName().replace(f"{prefix}_","") for ttree in tfile.GetListOfKeys()]
        treenames = np.unique(treenames)
        for i,treename in enumerate(treenames):
            
            print("\tTTree",i+1,"of",len(treenames))
            ttree = tfile.Get(treename)
            # From the tree name determine the binning
            treename = ttree.GetName()
            splitname = treename.split('_')
            name_list.append(splitname[0::3])
            min_list.append(np.array(splitname[1::3],dtype=float))
            max_list.append(np.array(splitname[2::3],dtype=float))
            center_list.append(0.5*(min_list[-1]+max_list[-1]))
            
            # Create an armada of histograms for each branch
            branches = [branch.GetName() for branch in ttree.GetListOfBranches()]
            branches = [branch for branch in branches if branch in list(pltfmt.keys())] # Only use branches declared in pltfmt
            
            hists={}
            df = ROOT.RDataFrame(treename,f"{input_dir}/{file}")
            hhisto1ds=[]
            for cut in cuts:
                histo1ds=[]
                for branch in branches:
                    NOW = datetime.datetime.now()
                    low = pltfmt[branch]["min"]
                    high = pltfmt[branch]["max"]
                    bins = pltfmt[branch]["bins"]
                    if(cut):
                        histo1ds.append(df.Filter(cut).Histo1D((f"{branch}_{treename}_{NOW}_{cut}","",bins,low,high),branch))
                    else:
                        histo1ds.append(df.Histo1D((f"{branch}_{treename}_{NOW}_all","",bins,low,high),branch))
                hhisto1ds.append(histo1ds)
            for cut,suffix,histo1ds in zip(cuts,datatypes,hhisto1ds):
                for branch,histo1d in zip(branches,histo1ds):
                    h = histo1d.GetValue()
                    h.Sumw2()
                    h.SetDirectory(0)
                    if(cut):
                        hists[f"{branch}_{suffix}"]=h
                    else:
                        hists[f"{branch}_all"]=h
            hlist.append(hists)
        #tfile.Close()
    

    min_list=np.array(min_list)
    max_list=np.array(max_list)
    center_list=np.array(center_list)
    
    unique_name_list = list(set([tuple(sublist) for sublist in name_list]))
    
    plots=[]
    for uname in unique_name_list:
        name=list(uname)
        plotdict={}
        plotdict["name"]=name
        BOOL =  [True if n == name else False for n in name_list]
        left = min_list[BOOL]
        right = max_list[BOOL]
        center = center_list[BOOL]
        hists = []
        for i,b in enumerate(BOOL):
            if b:
                hists.append(hlist[i])
        plotdict["leftEdge"]=left
        plotdict["rightEdge"]=right
        plotdict["centers"]=center
        plotdict["hists"]=hists
    
        plots.append(plotdict)
    
    full_plots=get_sorted_plots(plots,False)
    abbrev_plots=get_sorted_plots(plots,True)
    return full_plots, abbrev_plots

In [162]:
# collect()
# Reads from the "postprocess_binned" subdirectory
# This subdirectory typically contains a handful of .root files, starting with MC or nSidis, whose name specifies the binning of the TTrees within
#     - input_dir = Directory containing .root files
#     - skipEdges = Remove border bins (first and last for 1d binnings)
#     - datatype  = Flag for finding specific TTrees 
#     - cut       = Event-by-event cut

def collect(input_dir,skipEdges=False,datatype="nSidis",cut=""):
    
    assert(datatype=="nSidis" or datatype=="MC")
    
    name_list = []
    min_list = []
    max_list = []
    center_list = []
    hlist = []
    for file in os.listdir(f"{input_dir}/"):
        # Only analyze .root files
        if not file.endswith(".root"):
            continue
        if(not file.startswith(datatype)):
                continue
        
        # Open the .root file
        tfile = ROOT.TFile(f"{input_dir}/{file}","READ")
        print("Reading TFile",file)
        
        # Loop over all TTrees in the .root files
        treenames = [ttree.GetName().replace(f"{datatype}_","") for ttree in tfile.GetListOfKeys()]
        treenames = np.unique(treenames)
        for i,treename in enumerate(treenames):
            
            print("\tTTree",i+1,"of",len(treenames))
            ttree = tfile.Get(treename)
            # From the tree name determine the binning
            treename = ttree.GetName()
            splitname = treename.split('_')
            name_list.append(splitname[0::3])
            min_list.append(np.array(splitname[1::3],dtype=float))
            max_list.append(np.array(splitname[2::3],dtype=float))
            center_list.append(0.5*(min_list[-1]+max_list[-1]))
            
            # Create an armada of histograms for each branch
            branches = [branch.GetName() for branch in ttree.GetListOfBranches()]
            
            hists={}
            # Get hist min, max, and bins
#             for branch in branches:
                
#                 NOW = datetime.datetime.now()
#                 low = pltfmt[branch]["min"]
#                 high = pltfmt[branch]["max"]
#                 bins = pltfmt[branch]["bins"]
#                 h = ROOT.TH1F(f"{branch}_{treename}_{NOW}","",bins,low,high)
#                 h.Sumw2()
#                 ttree.Draw(f"{branch}>>{branch}_{treename}_{NOW}",cut,"goff")
#                 h.SetDirectory(0)
#                 hists[branch]=h
            df = ROOT.RDataFrame(treename,f"{input_dir}/{file}")
            if(cut):
                df = df.Filter(cut)
            histo1ds=[]
            for branch in branches:
                NOW = datetime.datetime.now()
                low = pltfmt[branch]["min"]
                high = pltfmt[branch]["max"]
                bins = pltfmt[branch]["bins"]
                histo1ds.append(df.Histo1D((f"{branch}_{treename}_{NOW}","",bins,low,high),branch))
            for branch,histo1d in zip(branches,histo1ds):
                h = histo1d.GetValue()
                h.Sumw2()
                h.SetDirectory(0)
                hists[branch]=h
            hlist.append(hists)
        #tfile.Close()
    

    min_list=np.array(min_list)
    max_list=np.array(max_list)
    center_list=np.array(center_list)
    
    unique_name_list = list(set([tuple(sublist) for sublist in name_list]))
    
    plots=[]
    for uname in unique_name_list:
        name=list(uname)
        plotdict={}
        plotdict["name"]=name
        BOOL =  [True if n == name else False for n in name_list]
        left = min_list[BOOL]
        right = max_list[BOOL]
        center = center_list[BOOL]
        hists = []
        for i,b in enumerate(BOOL):
            if b:
                hists.append(hlist[i])
        plotdict["leftEdge"]=left
        plotdict["rightEdge"]=right
        plotdict["centers"]=center
        plotdict["hists"]=hists
    
        plots.append(plotdict)
    
    plots=get_sorted_plots(plots,skipEdges)
    return plots

In [179]:
def make_binned_plots(params_data=0,params_MC=0,histname="",histname_data="",histname_MC="",OUTDIR="",skipEdges=False,dodiff=False,rescale_y=True):
    ROOT.gStyle.SetOptStat(0)
    # Make output directory
#     if not os.path.exists(outdir):
#         os.mkdir(outdir)
    
    for idx_data in range(len(params_data)):
        
        plot=params_data[idx_data]
        
        # Make output directory
        namelist = "dim{}".format(len(plot["name"]))
        for name in plot["name"]:
            namelist+=f"_{name}"
        outdir=OUTDIR+"/"+namelist+"/"+("ratio" if dodiff else "overlay")+"/"+("abbrev" if skipEdges else "full")+"/"+("rescale" if rescale_y else "default")
        
        if not os.path.exists(outdir):
            os.makedirs(outdir)
        
        dim=len(plot["leftEdge"][0])
        if(dim>2):
            print("Can only do Mdiphoton plotting for dim<=2...Aborting...")
            break



        # Set global parameters
        cleft=-20
        cright=20
        cup=1
        cdown=-1
        xaxis_h = -0.8
        xaxis_l = -15
        xaxis_r = 14
        padtop=0.8
        padbot=0.2

        padtopscale=(cup-cdown)*padtop+cdown
        padbotscale=(cup-cdown)*padbot+cdown

        #Create TCanvas
        c=ROOT.TCanvas("c","c",1200,600)
        c.Range(cleft,cdown,cright,cup)


        # Get xmin and xmax
        leftEdge=np.unique(plot["leftEdge"][:,0])
        rightEdge=np.unique(plot["rightEdge"][:,0])
        xmin=leftEdge[0]
        xmax=rightEdge[-1]
        if(dim==2):
            bottomEdge=np.unique(plot["leftEdge"][:,1])
            topEdge=np.unique(plot["rightEdge"][:,1])
            ymin = bottomEdge[0]
            ymax = topEdge[-1]

        # Set number of xbins
        nxbins=len(leftEdge)
        # Set number of ybins
        nybins=1
        if(dim==2):
            nybins=len(topEdge)

        # Set TPad box size
        boxheight=(padtop-padbot)/nybins
        boxheightscale=(padtopscale-padbotscale)/nybins

        # Get bins
        bins = sorted(np.unique(np.concatenate([leftEdge,rightEdge])))

        # Create axes
        # If dimension == 2, make a y axis as well
        xaxis=ROOT.TGaxis(xaxis_l,xaxis_h,xaxis_r,xaxis_h,xmin,xmax,510,"<")
        xaxis.SetTitle(plot["name"][0])
        xaxis.Draw()

        if(dim==2):
            yaxis=ROOT.TGaxis(xaxis_l-1.75,padbotscale,xaxis_l-1.75,padtopscale,ymin,ymax,508,"")
            if(plot["name"][1]=="Mh"):
                yaxis.SetTitle("M_{#pi^{+}#pi^{0}}[GeV]")
            else:
                yaxis.SetTitle(plot["name"][1])
            yaxis.Draw("same")

        # Scale bins to axes
        bins_scaled =(( (bins - np.min(bins)) * (xaxis_r - (xaxis_l)) / (np.max(bins) - np.min(bins)) + (xaxis_l)) - cleft)/ (cright-cleft) 
        bins_scaled_v2 =( (bins - np.min(bins)) * (xaxis_r - (xaxis_l)) / (np.max(bins) - np.min(bins)) + (xaxis_l))
        
        
        # Find the ymax for the fits
        ymin=0
        ymax=0
        if(rescale_y):
            for hh in plot["hists"]:
                h=hh[histname_data]
                if(h.GetMaximum()>ymax):
                    ymax=h.GetMaximum()*1.1
        if(dodiff):
            ymin=0
            ymax=3
        
        # For each mdiphoton
        xxaxis=[]
        yyaxis=[]
        lines=[]
        i=0
        inc=0
        ix,iy=0,0
        for hh_idx in range(len(plot["hists"])):
            hh=plot["hists"][hh_idx]
            if(params_MC!=0):
                for idx_mc in range(len(params_MC)):
                    # Since the index of the MC plot may not be aligned with the index of the data plot
                    # we have this extra code to match them
                    if(params_MC[idx_mc]["name"]==params_data[idx_data]["name"]):
                        h_mc=params_MC[idx_mc]["hists"][hh_idx][histname_MC]
                        break
            h=hh[histname_data]
            if(ix==nxbins):
                ix=0
                iy+=1
                
            xxaxis.append(ROOT.TGaxis(bins_scaled_v2[ix],padbotscale,bins_scaled_v2[ix+1],padbotscale,h.GetXaxis().GetXmin(),h.GetXaxis().GetXmax(),506,"S"))
            yyaxis.append(ROOT.TGaxis(bins_scaled_v2[ix],padbotscale+boxheightscale*iy,bins_scaled_v2[ix],padbotscale+boxheightscale*(iy+1),ymin,ymax,505,"S"))
            xxaxis[i].SetLabelSize(0.03)
            yyaxis[i].SetLabelFont(42)
            yyaxis[i].SetLabelSize(0)
            yyaxis[i].SetTickLength(0.02)
            if(ix!=nxbins-1):
                xxaxis[i].SetLabelSize(0)
            tpad = ROOT.TPad("","",bins_scaled[ix],padbot+boxheight*iy,bins_scaled[ix+1],padbot+boxheight*(iy+1))
            tpad.Draw("same")
            tpad.cd()
            tpad.SetLeftMargin(0)
            tpad.SetRightMargin(0)

            tpad.SetTopMargin(0)
            tpad.SetBottomMargin(0)
            
            
            h.SetTitle("")
            h.GetXaxis().SetLabelSize(0)
            h.GetXaxis().SetNdivisions(0)
            h.GetYaxis().SetNdivisions(0)
            h.SetLineColor(1)
            if(dodiff):
                if(h_mc.Integral()!=0):
                    h_mc.Scale(h.Integral()/h_mc.Integral())
                h.Divide(h_mc)
                h.Draw("hist E1")
                lines.append(ROOT.TLine(h.GetXaxis().GetXmin(),1,h.GetXaxis().GetXmax(),1))
                lines[i].SetLineStyle(7)
                lines[i].SetLineColor(8)
                lines[i].Draw("same")
            else:
                h.Draw("hist")
                if(params_MC!=0):
                    h_mc.SetLineColor(2)
                    if(h_mc.Integral()!=0):
                        h_mc.Scale(h.Integral()/h_mc.Integral())
                    h_mc.Draw("hist same")
            # If we do not want to rescale the y, at least determine the ymax from the MC and data
            if(dodiff==False and rescale_y==False):
                ymax=1.05*np.amax([h.GetMaximum(),h_mc.GetMaximum()])
            h.GetYaxis().SetRangeUser(ymin,ymax)
            
            c.cd()
            xxaxis[i].SetLabelFont(42)
            xxaxis[i].SetTickLength(0.06)
            xxaxis[i].Draw("same")
            if(dodiff):
                if(ix==0 and iy==0):
                    yyaxis[i].SetLabelSize(0.03)
                    yyaxis[i].SetTickLength(0.06)
            yyaxis[i].Draw("same")
            
            i=i+1
            inc=inc+1
            ix+=1
            
        latex=ROOT.TLatex()
        latex.SetTextFont(42)
        latex.DrawLatexNDC(0.87,padbot-0.02,pltfmt[histname]["xtitle"])
        
        if(params_MC!=0 and dodiff==False):
            legend=ROOT.TLegend(0.86,padbot+0.05,0.99,padbot+0.15)
            legend.AddEntry(h,"Data","l")
            legend.AddEntry(h_mc,"Monte Carlo","l")
            legend.SetBorderSize(0)
            legend.Draw("same")
        
        subdir = ""
        for n in plot["name"]:
            subdir+=n+"_"
        if(skipEdges):
            subdir+="abbrev"
        else:
            subdir+="full"
        if(dodiff):
            subdir+="_diff"
        # Create subdirectory to store plots
#         if not os.path.exists(f"{outdir}/{subdir}"):
#             os.mkdir(f"{outdir}/{subdir}")
        c.SaveAs(f"{outdir}/{histname}.png")

In [177]:
# input_dir = "/volatile/clas12/users/gmat/clas12analysis.sidis.data/rga/ML/catboost/binned_pipluspi0/test1d"
# output_dir = "/work/clas12/users/gmat/scipio/macros/analysis/plots"
# plots=collect(input_dir,True)

Reading TFile z_binned.root
	TTree 1 of 8
	TTree 2 of 8
	TTree 3 of 8
	TTree 4 of 8
	TTree 5 of 8
	TTree 6 of 8
	TTree 7 of 8
	TTree 8 of 8
Reading TFile x_binned.root
	TTree 1 of 8
	TTree 2 of 8
	TTree 3 of 8
	TTree 4 of 8
	TTree 5 of 8
	TTree 6 of 8
	TTree 7 of 8
	TTree 8 of 8
Reading TFile Mh_binned.root
	TTree 1 of 9
	TTree 2 of 9
	TTree 3 of 9
	TTree 4 of 9
	TTree 5 of 9
	TTree 6 of 9
	TTree 7 of 9
	TTree 8 of 9
	TTree 9 of 9


In [180]:
# make_binned_plots(plots,"Mgg",output_dir,True)

Info in <TCanvas::Print>: png file /work/clas12/users/gmat/scipio/macros/analysis/plots/z_abbrev/Mgg.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c
Info in <TCanvas::Print>: png file /work/clas12/users/gmat/scipio/macros/analysis/plots/Mh_abbrev/Mgg.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c
Info in <TCanvas::Print>: png file /work/clas12/users/gmat/scipio/macros/analysis/plots/x_abbrev/Mgg.png has been created
